For our group project client criteria 1： **Break and Holiday schedule <or = Regular schedule**. At that time, there was no in-depth analysis of the office situation in this range. Therefore, when conducting market research, it is necessary to classify offices with similar consumption and estimate their office types and consumption methods. This is what I do in my individual project.

In fact, I am not very talented in programming, so I did not use pandas to do very diversified analysis. Part of the code comes from the contributions of groupwork members.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
meta = pd.read_csv('../input/building-data-genome-project-v1/meta_open.csv',index_col='uid') 
meta.head()

In [ ]:
meta_phoenix=meta.loc[meta['timezone']=='America/Phoenix']
meta_phoenix

In [ ]:
meta_office=meta_phoenix[meta_phoenix['primaryspaceusage']=='Office']
meta_office.head()

In [ ]:
meta_office.reset_index()

In [ ]:
meta_office['uid'] = meta_office.index

In [ ]:
Officename = meta_office['uid'].tolist()


In [ ]:
data_list_normalized = []
for officename in Officename:
    
  rawdata = pd.read_csv("../input/building-data-genome-project-v1/"+officename+".csv", parse_dates=True, index_col='timestamp')
  floor_area = meta.loc[officename]["sqm"]

  normalized_data = rawdata/floor_area

  data_list_normalized.append(normalized_data)
normalize_office = pd.concat(data_list_normalized,axis=1)
normalize_office

In [ ]:
schedule = pd.read_csv('../input/building-data-genome-project-v1/schedule2.csv',header = None, parse_dates=True) 
schedule.columns = ['timestamp','schedule']
schedule['timestamp']=pd.to_datetime(schedule['timestamp'],format='%Y-%m-%d')
schedule.set_index(['timestamp'],inplace = True)
schedule

In [ ]:
schedule['schedule'].value_counts()

In [ ]:
#set the same column of 'Date'
normalize_office['Time'] = normalize_office.index.map(lambda t: t.time())
normalize_office['Date'] = normalize_office.index.map(lambda t: t.date())
schedule['Date'] = schedule.index.map(lambda t: t.date())

In [ ]:
#combine normalize_office and schedule
normalize_office_schedule = pd.merge(schedule, normalize_office, how='left', on='Date')
normalize_office_schedule

In [ ]:
#break.max>regular.max,holiday.max>regular.max
normalized_result = normalize_office_schedule.groupby(['schedule','Time']).mean()
tmp = normalized_result.groupby(['schedule'])
tmp.max()

In [ ]:
#select clients
clients1 = []
for i in Officename:
  if tmp.max().loc['Break'][i]>=tmp.max().loc['Regular'][i] or tmp.max().loc['Holiday'][i]>=tmp.max().loc['Regular'][i]: 
    clients1.append(i)
clients1

In [ ]:
#Very important!!!!!!!!
normalized_result = normalize_office_schedule.groupby(['schedule','Time']).mean()
df_clients1 = normalized_result[clients1]
df_clients1.plot.area(figsize=(25,8),stacked=False)

From this picture, we can make a rough classification analysis of offices with similar consumption patterns. The next step is to compile the plot of office_Alannah, and the rest are analyzed with pictures (all screenshots after being generated on this page) to reduce the corresponding time of Kaggle running.

In [ ]:
clients1

There are three typical and different consumption patterns in these nine offices, and I will analyze them one by one next.

In [ ]:
def clients1_pivot_plot(clients1):
  reset_index = df_clients1[[clients1]].reset_index()
  pivot = reset_index.pivot(index='Time',columns='schedule',values=clients1)
  pivot.plot(figsize=(18,8)) 
  plt.title('24 load profile')
  plt.tick_params(labelsize=20)
  return plt.ylabel('Energy Consumption')
clients1_pivot_plot('Office_Alannah')

In [ ]:
clients1_pivot_plot('Office_Amelia')

The first type: office Alannah and Amelia have similar consumption patterns. The first peak appeared at around 5:30, and consumption changed rapidly, as can be seen from the slope. Then the second peak of electricity consumption was around 14:00. 

The consumption waveform structure of the two offices is very similar, and the electricity consumption during working hours follows the rule of Summer>Regular>Holiday>Break.

What's interesting is that at the first peak, the two offices showed a state that the consumption of holiday and break was greater than regular. It can be inferred that the company's working mode is that regular time works according to normal working hours. During holidays or breaks, they will work overtime in the morning because they need to improve efficiency. Electricity consumption has been very low since the morning, so the duties performed by the office should be management of transportation and delivery. Overtime work in the early morning is likely to require expedited delivery, after which the employees can return to their holiday life normally.

In [ ]:
clients1_pivot_plot('Office_Angelina')

In [ ]:
clients1_pivot_plot('Office_Alyson')

The second type: office_Angelina and Alyson have unique consumption patterns. The first peak occurred between 5:00-5:30, and the consumption changes during the rest of the time were not very obvious. It is worth noting that the consumption patterns of Break and Holiday are the same, and the consumption patterns of Regular and Summer are the same. 

And the electricity consumption of the two offices as a whole follows the rule of Break>Holiday>Regular>Summer.

Obviously, a normal office consumption in Break and Holiday is lower than Regular. The consumption of this type of office seems abnormal, so it can be inferred that this office is a service-type holiday-driven profit model. During holidays, the workload is increased as needed and the power consumption of the equipment is increased. In the off-season (Regular), electricity consumption is not so frequent.

In [ ]:
clients1_pivot_plot('Office_Allyson')

In [ ]:
clients1_pivot_plot('Office_Ava')

In [ ]:
clients1_pivot_plot('Office_Abbey')

The third type: I call it a regular office type. The consumption performance of Ava, Allyson and Abbey all belong to this mode. The first peak occurs around 9:00, which is the prime time for work, and the second small peak will follow after sunset. All electricity consumption is concentrated in regular working hours (7:30-21:00). 

Office Ava and Allyson do not seem to have a strict holiday concept, because the electricity consumption under different schedules is almost similar. Abbey is a little different from them. At least during Holiday, the employees seem to take a break because the overall electricity consumption is maintained at a very low level. If it were me, I might prefer to go to Abbey (LOL).

After investigation, the Phoenix City of the United States is in the leading position in the field of solar energy and semiconductors in the United States. Perhaps future energy-saving projects can be combined with solar power for intelligent power control.